In [28]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from PIL import Image

In [31]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

train_ds = tf.keras.utils.image_dataset_from_directory(
    "datasets/train",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="binary"
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    "datasets/val",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="binary"
)

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.shuffle(1000).prefetch(AUTOTUNE)
val_ds = val_ds.prefetch(AUTOTUNE)


Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.


In [32]:

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.shuffle(1000).prefetch(AUTOTUNE)
val_ds = val_ds.prefetch(AUTOTUNE)


In [33]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=IMG_SIZE + (3,),
    include_top=False,
    weights="imagenet"
)
base_model.trainable = False

data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

inputs = keras.Input(shape=IMG_SIZE + (3,))
x = data_augmentation(inputs)
x = tf.keras.applications.mobilenet_v2.preprocess_input(x)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model = keras.Model(inputs, outputs)

In [34]:

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

history = model.fit(
    train_ds,
    epochs=5,
    validation_data=val_ds
)

Epoch 1/5


625/625 ━━━━━━━━━━━━━━━━━━━━ 131s 192ms/step - accuracy: 0.9608 - loss: 0.1078 - val_accuracy: 0.9858 - val_loss: 0.0400
Epoch 2/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 101s 154ms/step - accuracy: 0.9747 - loss: 0.0661 - val_accuracy: 0.9876 - val_loss: 0.0351
Epoch 3/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 94s 146ms/step - accuracy: 0.9772 - loss: 0.0612 - val_accuracy: 0.9878 - val_loss: 0.0326
Epoch 4/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 101s 157ms/step - accuracy: 0.9796 - loss: 0.0566 - val_accuracy: 0.9878 - val_loss: 0.0337
Epoch 5/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 100s 155ms/step - accuracy: 0.9783 - loss: 0.0576 - val_accuracy: 0.9898 - val_loss: 0.0316


In [36]:
loss, acc = model.evaluate(val_ds)
print(f"Final Validation Accuracy: {acc*100:.2f}%")

157/157 ━━━━━━━━━━━━━━━━━━━━ 17s 108ms/step - accuracy: 0.9898 - loss: 0.0316
Final Validation Accuracy: 98.98%
